Animation showing line-of-sight from Earth to other planets, illustrating the cause of retrograde motion. Retrograde motion is illustrated with a red line. The beginning and ending dates of retrograde motion and the date of opposition are indicated with text.

I wrote this script in Carnets app on my iPad ( https://apps.apple.com/us/app/carnets/id1450994949 ) and in Anaconda on my Mac ( https://www.anaconda.com ). There's a quirk in the interpreter (in both development environments) that causes the code to fail with error message `AttributeError: 'NoneType' object has no attribute 'remove_callback'`. I just ignore it and it runs the second time.

This script uses Skyfield api ( https://rhodesmill.org/skyfield/planets.html ).

Although I probably can't help you debug your program I would love to hear if you use this animation (or derivative) in your astronomy lessons or observations.

Good luck,

Stephen Shadle 🌌

swshadle@gmail.com

In [1]:
# %pip install --upgrade ipykernel

In [2]:
# # Suppress warnings. Comment this out if you wish to see the warning messages
# import warnings
# warnings.filterwarnings('once') #'ignore'
# print('warnings suppressed')

In [18]:
# use "%pip install" to load skyfield api once then comment the line out:
###### %pip install skyfield
from skyfield.api import load
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import MaxNLocator
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FFMpegWriter, PillowWriter
import matplotlib.animation as animations
from datetime import datetime
%matplotlib notebook

start_time = datetime.now()

# constructor class for making planets
class planets:
    def __init__(self, name, marker=None, color=None, markersize=None, trailmax=None):
        self.name       = name
        self.marker     = marker if marker is not None else 'o'
        self.color      = color if color is not None else 'k'
        self.markersize = markersize if markersize is not None else 1

# create a list of solar system objects
plist = []

# append instances to list
plist.append(planets(name='Sun',     color='yellow',          markersize=7, marker='*'))
plist.append(planets(name='Mercury', color='burlywood',       markersize=2))
plist.append(planets(name='Venus',   color='blanchedalmond',  markersize=4))
plist.append(planets(name='Earth',   color='mediumturquoise', markersize=4))
plist.append(planets(name='Mars',    color='lightcoral',      markersize=3))
# plist.append(planets(name='Jupiter', color='burlywood',       markersize=6))
# plist.append(planets(name='Saturn',  color='seashell',        markersize=5))
# plist.append(planets(name='Uranus',  color='paleturquoise',   markersize=5))
# plist.append(planets(name='Neptune', color='royalblue',       markersize=5))
# plist.append(planets(name='Pluto',   color='lightsalmon',     markersize=2))

# load skyfield ephemeris table from local directory (see https://rhodesmill.org/skyfield/planets.html)
eph = load('de421.bsp')
sun     = eph[plist[0].name]
mercury = eph[plist[1].name]
venus   = eph[plist[2].name]
earth   = eph[plist[3].name]
mars    = eph[plist[4].name]
# jupiter = eph['Jupiter barycenter']
# saturn  = eph['Saturn barycenter']
# urаnus  = eph['Uranus barycenter']
# neptune = eph['Neptune barycenter']
# pluto   = eph['Pluto barycenter']

ts = load.timescale()

hours = (3*365+1)*24 # how long the animation should run (number of hours of data from ephemeris table)

interval = 24        # number of hours to skip forward for each frame of the animation
dpi = 120            # dots per inch for the image
hw = 3.0             # height and width of the figure, in AU. 3.0 works for the inner planets through mars

# ephemeris table de421.bsp is only valid through 2053 Oct 9 (Julian date 2471184.5)
numremoved = 0        # zero data points removed due to valid date range (so far!)

start_y = start_time.year  # 2020 (use numbers here to select a fixed start date rather than today's date)
start_m = start_time.month # 5
start_d = start_time.day   # 1

t = ts.utc(start_y,start_m,start_d,range(hours))
while len(t)>1 and t[-1].tt>2471184.5:
    t=t[:-1]
    numremoved += 1

if numremoved:
    print(f'removed {numremoved} frames')
    hours -= numremoved
    t  = ts.utc(start_y,start_m,start_d,range(hours))

# assert an error if there is no remaining valid data
assert t[-1].tt<2471184.5, 'Dates are out of range'
assert hours>0, f'too few hours selected: {hours}'

sun_p     = eph[plist[0].name].at(t).ecliptic_position().au
mercury_p = eph[plist[1].name].at(t).ecliptic_position().au
venus_p   = eph[plist[2].name].at(t).ecliptic_position().au
earth_p   = eph[plist[3].name].at(t).ecliptic_position().au
# moon_p    = moon.at(t).ecliptic_position().au
mars_p    = eph[plist[4].name].at(t).ecliptic_position().au
# jupiter_p = jupiter.at(t).ecliptic_position().au
# saturn_p  = saturn.at(t).ecliptic_position().au
# uranus_p  = urаnus.at(t).ecliptic_position().au
# neptune_p = neptune.at(t).ecliptic_position().au
# pluto_p   = pluto.at(t).ecliptic_position().au

ps = [sun_p,
      mercury_p,
      venus_p,
      earth_p,
#       moon_p,
      mars_p,
#       jupiter_p,
#       saturn_p,
#       uranus_p,
#       neptune_p,
#       pluto_p,
     ]

maxsize = [-hw, hw]

# with plt.xkcd(): # uncomment (and indent the rest of the script) if you want the figure to be in xkcd style (see https://matplotlib.org/xkcd/examples/showcase/xkcd.html)
fig = plt.figure(figsize=(6.6, 5), dpi=dpi)

ax  = fig.add_subplot(1, 1, 1,
#                        projection ='3d',
                      )

plt.subplots_adjust(left=-0.25, right=1) # moves the plot to reduce the left margin

# some background stars (from https://github.com/zingale/astro_animations/blob/master/solar_system_motion/retrograde/retrograde.py)
import random
N = 40
xpos = []
ypos = []
starbox = [2.6, 3.0]
for s in range(N):
    # right
    xpos.append(random.uniform( starbox[0], starbox[1]))
    ypos.append(random.uniform(-starbox[1], starbox[1]))

    # top
    xpos.append(random.uniform(-starbox[1], starbox[1]))
    ypos.append(random.uniform( starbox[0], starbox[1]))

    # left
    xpos.append(random.uniform(-starbox[0],-starbox[1]))
    ypos.append(random.uniform(-starbox[1], starbox[1]))

    # bottom
    xpos.append(random.uniform(-starbox[1], starbox[1]))
    ypos.append(random.uniform(-starbox[0],-starbox[1]))

# draw some random background stars
for s in range(N):
    plt.scatter([xpos[s]], [ypos[s]], s=20, marker=(5,1), color="grey")
plt.show()

dots   = []
trails = []

for i,p in enumerate(ps):
    x, y, z = p # z is ignored in 2D plots

    newdot, = plt.plot([x[0]],
                       [y[0]],
#                        [z[0]], # not used in a 2D plot
                       marker=plist[i].marker,
                       ms=plist[i].markersize,
                       label=plist[i].name,
                       color=plist[i].color,
                      linestyle = 'None',
                      )
    dots.append(newdot)
    
    newtrail, = plt.plot(x[:1],
                         y[:1],
#                          z[:1], # not used in a 2D plot
                         color=plist[i].color,
                         linewidth=0.01,
#                          marker='.',
#                          ms=0.01,
#                          linestyle = 'None',
                        )

    trails.append(newtrail)

mercury_x, mercury_y, mercury_z = ps[1]
venus_x, venus_y, venus_z       = ps[2]
earth_x, earth_y, earth_z       = ps[3]
mars_x, mars_y, mars_z          = ps[4]

# get slopes for line-of-sight lines from earth through other planets
mars_slope     = (earth_y[0] -    mars_y[0])/(earth_x[0] -    mars_x[0])
venus_slope    = (earth_y[0] -   venus_y[0])/(earth_x[0] -   venus_x[0])
mercury_slope  = (earth_y[0] - mercury_y[0])/(earth_x[0] - mercury_x[0])

# endpoint of line-of-sight line should be to the right of the figure if the planet is to the right of earth & vice versa
if earth_x[0]>mars_x[0]:
    mars_xpt = -3.5
else:
    mars_xpt = 3.5

mars_ypt = earth_y[0] + mars_slope*(mars_xpt - earth_x[0])

if earth_x[0]>venus_x[0]:
    venus_xpt = -3.5
else:
    venus_xpt = 3.5

venus_ypt = earth_y[0] + venus_slope*(venus_xpt - earth_x[0])

if earth_x[0]>mercury_x[0]:
    mercury_xpt = -3.5
else:
    mercury_xpt = 3.5

mercury_ypt = earth_y[0] + mercury_slope*(mercury_xpt - earth_x[0])

eclat, mars_eclon, ecd = earth.at(t).observe(mars).ecliptic_latlon()
mars_eclondel = mars_eclon.radians[1:] - mars_eclon.radians[:-1]

# combine into an array with 2 columns. this keeps dates and motion in synch during the data cleaning
mars_data2d = np.stack((mars_eclondel, t[:-1]), axis=1) # make a 2d data structure where column 0 is delta ecliptic longitude and column 1 is dates

# data cleaning
def reject_outliers(data, m=2.):
    d = np.abs(data[:,0] - np.median(data[:,0]))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

filtered_mars_data2d = reject_outliers(mars_data2d,200.)

# identify where mars is in prograde (which we will mark with a grey line) and retrograde (orange-red line)
mars_prograde = filtered_mars_data2d[:,0] >= 0.0

mars_motion_chg = np.where(mars_prograde[:-1] != mars_prograde[1:])[0]+1

# make a structured array (events) to hold a list of notable events (begin/end retrograde & opposition) for mars and, later, other planets
dtype = [('Frame', np.int32), ('Date', (np.str_, 10)), ('Text', (np.str_, 25))]
events = np.array([], dtype=dtype)
display_events = np.array([], dtype=plt.Text)

# construct list of events
opposition = 0
for i in mars_motion_chg:
    if not mars_prograde[i]:
        opposition = i
        events = np.concatenate((events, np.array([(i,'{:%Y-%m-%d}'.format(filtered_mars_data2d[i,1].utc_datetime()),"Mars begins retrograde")], dtype=dtype)), axis=0)
#         print('Mars begins retrograde\n{:%Y-%m-%d} frame #{}'.format(filtered_mars_data2d[i,1].utc_datetime(), i))
    else:
        if opposition != 0:
            opposition = (opposition + i) // 2 # integer floor division
            events = np.concatenate((events, np.array([(opposition,'{:%Y-%m-%d}'.format(filtered_mars_data2d[opposition,1].utc_datetime()),"Mars at opposition")], dtype=dtype)), axis=0)
#             print('Mars at opposition\n{:%Y-%m-%d} frame #{}'.format(filtered_mars_data2d[opposition,1].utc_datetime(), opposition))
        events = np.concatenate((events, np.array([(i,'{:%Y-%m-%d}'.format(filtered_mars_data2d[i,1].utc_datetime()),"Mars ends retrograde")], dtype=dtype)), axis=0)
#         print('Mars ends retrograde\n{:%Y-%m-%d} frame #{}'.format(filtered_mars_data2d[i,1].utc_datetime(), i))        

# next add events for Jupiter, Saturn, etc. before sorting
# ... #

# sort events by Frame number. not strictly necessary with just data for mars.
events = np.sort(events, order='Frame')

# construct display_events after sorting events
text_x = 1.02
text_y = 0.65
text_y_delta = 0.07
fontsize=8
# opposition = 0
for i, frame in enumerate(events):
    display_events = np.concatenate((
                                     display_events,
#                                      np.array([(ax.text(text_x, text_y, '{}\n{} frame #{}'.format(events['Text'][i], events['Date'][i], events['Frame'][i]), 
                                     np.array([(ax.text(text_x, text_y, '{}\n{}'.format(events['Text'][i], events['Date'][i]), 
                                                        transform=ax.transAxes,
                                                        fontsize=8,
                                                        visible=False))]) # initially, events are hidden. they are revealed after the event has passed
                                    ))
    text_y -= text_y_delta
        
# store an array of colors (grey or orangered)
mars_linecolor = ['grey' if mars_prograde[i] else 'orangered' for i in range(len(mars_prograde))]

# create the line that shows line-of-sight through mars at the first (0th) data point. we'll do the rest in the animate function
mars_line,  = plt.plot([earth_x[0], mars_xpt], [earth_y[0], mars_ypt], "--",
                   linewidth=0.5) # line connecting earth and mars
mars_line.set_color(mars_linecolor[0])

eclat, venus_eclon, ecd = earth.at(t).observe(venus).ecliptic_latlon()
venus_eclondel = venus_eclon.radians[1:] - venus_eclon.radians[:-1]
venus_prograde = venus_eclondel >= 0.
venus_linecolor = ['grey' if venus_prograde[i] else 'orangered' for i in range(len(venus_prograde))]
venus_line,  = plt.plot([earth_x[0], venus_xpt], [earth_y[0], venus_ypt], "--",
                   linewidth=0.5) # line connecting earth and venus
venus_line.set_color(venus_linecolor[0])

eclat, mercury_eclon, ecd = earth.at(t).observe(mercury).ecliptic_latlon()
mercury_eclondel = mercury_eclon.radians[1:] - mercury_eclon.radians[:-1]
mercury_prograde = mercury_eclondel >= 0.
mercury_linecolor = ['grey' if mercury_prograde[i] else 'orangered' for i in range(len(mercury_prograde))]
mercury_line,  = plt.plot([earth_x[0], mercury_xpt], [earth_y[0], mercury_ypt], "--",
                   linewidth=0.5) # line connecting earth and mercury
mercury_line.set_color(mercury_linecolor[0])

ax.set_xlim(maxsize)
ax.set_ylim(maxsize)
# ax.set_zlim(maxsize) # not used in a 2D plot

ax.set_ylabel(None, 
              labelpad = 14,
             ) # elevation
ax.set_xlabel('distance in AU',
#               labelpad = 14,
             )
# ax.set_zlabel(None)   # not used in a 2D plot

# set up animation of axis ticks and labels
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
# ax.zaxis.set_major_locator(MaxNLocator(integer=True))

ticks =  ax.get_yticks()
# ticks = ticks[1:] # compensates for an error in some versions of the 3d view system. might not be needed (you might have better results if you comment or uncomment this line)

# set labels to integers with absolute values (representing distance from sun in AU)
ax.set_xticks(ax.get_xticks().tolist())
ax.set_yticks(ax.get_yticks().tolist())

ax.set_xticklabels([int(abs(tick)) for tick in ticks])
ax.set_yticklabels([int(abs(tick)) for tick in ticks])
# ax.set_zticklabels([int(abs(tick)) for tick in ticks])

handles, labels = ax.get_legend_handles_labels()
plt.style.use(['dark_background'])
plt.rcParams['figure.facecolor'] = 'black'
plt.legend(bbox_to_anchor=(1.3, 1.0), # sets the legend outside the figure
               handles=handles,       # our updated list of handles
               loc='upper right',
               ncol=1,
               fontsize='medium',
               borderaxespad=0.0,
               shadow=True,
              )

ax.set_aspect('equal') # makes circles look like circles

if interval == 24:
    timeperframe = '1 day'
elif interval > 24:
    if interval%24==0:
        timeperframe = f'{interval//24} days'
    else:
        timeperframe = '{:^.1f} days'.format(interval/24)
elif interval == 1:
    timeperframe = '1 hr'
else:
    timeperframe = f'{interval} hrs'

# placement 0, 0 would be bottom-left. 1, 1 would be top-right.
text1 = ax.text(0.0, 1.05, '', transform=ax.transAxes) # date/time for each frame
text2 = ax.text(0.7, 1.05, f'{timeperframe}/frame', transform=ax.transAxes) # hours or days per frame of animation

# init function doesn't do anything currently. it's there if any setup is needed before starting the animation
def init():
    return dots, trails, text1, text2, display_events

def animate(hour):
    pctleft = (hours-hour-hours%interval)/hours
    pctzoom = 0.025+1.0*(1-np.sin((np.pi/2)*(pctleft)))
#     text1.set_text('{:%Y-%m-%d %H:%M} frame #{}'.format(t[hour].utc_datetime(), hour))
    text1.set_text('{:%Y-%m-%d %H:%M}'.format(t[hour].utc_datetime()))
    ticks =  ax.get_yticks()

#   use decimal axis labels when zoomed in
    if pctzoom<0.0245:                  # :^.1f
        ax.set_xticklabels([f'{abs(tick):.1f}' for tick in ticks])
        ax.set_yticklabels([f'{abs(tick):.1f}' for tick in ticks])
#         ax.set_zticklabels([f'{abs(tick):.1f}' for tick in ticks])
    else:
        ax.set_xticklabels([int(abs(tick)) for tick in ticks])
        ax.set_yticklabels([int(abs(tick)) for tick in ticks])
#         ax.set_zticklabels([int(abs(tick)) for tick in ticks])

# update the postion and trails for each planet
    for i,p in enumerate(ps):
        x, y, z = p # z is ignored in 2D plots
        dots[i].remove()
        dots[i], = plt.plot([x[hour]], [y[hour]],
#                             [z[hour]],
                            marker=plist[i].marker,
                            ms=plist[i].markersize,
                            color=plist[i].color,
                            linestyle = 'None',
                        )

        trails[i].remove()
        trails[i], = plt.plot(x[:hour], y[:hour],
#                               z[:hour],
                              color=plist[i].color,
                              linewidth= 0.8,
                              alpha = 1,
                        )

    # draw a line connecting earth and another planet and extending a bit further out
    mars_slope    = (earth_y[hour] -    mars_y[hour])/(earth_x[hour] -    mars_x[hour])
    venus_slope   = (earth_y[hour] -   venus_y[hour])/(earth_x[hour] -   venus_x[hour])
    mercury_slope = (earth_y[hour] - mercury_y[hour])/(earth_x[hour] - mercury_x[hour])

    if earth_x[hour]>mars_x[hour]:
        mars_xpt = -3.5
    else:
        mars_xpt = 3.5

    mars_ypt = earth_y[hour] + mars_slope*(mars_xpt - earth_x[hour])

    if earth_x[hour]>venus_x[hour]:
        venus_xpt = -3.5
    else:
        venus_xpt = 3.5

    venus_ypt = earth_y[hour] + venus_slope*(venus_xpt - earth_x[hour])

    if earth_x[hour]>mercury_x[hour]:
        mercury_xpt = -3.5
    else:
        mercury_xpt = 3.5

    mercury_ypt = earth_y[hour] + mercury_slope*(mercury_xpt - earth_x[hour])

    mars_line.set_data((earth_x[hour], mars_xpt), (earth_y[hour], mars_ypt))
    mars_line.set_color(color=mars_linecolor[hour])

    venus_line.set_data((earth_x[hour], venus_xpt), (earth_y[hour], venus_ypt))
    venus_line.set_color(color=venus_linecolor[hour])

    mercury_line.set_data((earth_x[hour], mercury_xpt), (earth_y[hour], mercury_ypt))
    mercury_line.set_color(color=mercury_linecolor[hour])

# use set_visible to turn on events after the "hour" when they have occurred
    for i, n in enumerate(display_events):
        display_events[i].set_visible(events['Frame'][i]<hour)
        
    return dots, trails, text1, text2, display_events

# lambda function prints the progress to the console while rendering the animation.
progress_callback = lambda i, n: print(f'Saving frame {i+1} of {n}')
# plt.show() # normally you need to tell matplotlib and pyplot to show the plot. this isn't needed when using FuncAnimation
animation = FuncAnimation(fig, animate, 
                          frames=range(0,hours,interval), # run the animation for all days represented
                          interval=10, blit=False, repeat=False, 
                          repeat_delay=5,
                          init_func=init)

filename = f"retrograde multiple.{dpi} dpi.{hours} pts.{interval} interval"

# please update the 'artist' metadata for your animation file by putting your name as the artist:
artist = 'Stephen Shadle'

if FFMpegWriter.isAvailable():
    print('FFMpegWriter is available')
    writer = FFMpegWriter(fps=15, metadata=dict(artist=artist), bitrate=1800)
# comment the following line to run animation without saving the file
    animation.save(filename=f'{filename}.mp4', writer=writer, progress_callback=progress_callback)
else:
    print('FFMpegWriter is not available')
    if PillowWriter.isAvailable():
        print('PillowWriter is available')
        Writer = animations.writers['pillow']
        writer = Writer(fps=60, metadata=dict(artist=artist), bitrate=1800,)
# comment the following line to run animation without saving the file
        animation.save(filename=f'{filename}.gif', writer=writer, dpi=dpi)
    else:
        print('PillowWriter is not available')

# elapsed time function
# PLACE THE FOLLOWING AT THE TOP
# from datetime import datetime
# from time import sleep
# start_time = datetime.now()

time_elapsed = datetime.now() - start_time

da, remainder  = divmod(time_elapsed.total_seconds(), 24*3600)
hrs, remainder = divmod(remainder, 3600)
mins, secs = divmod(remainder, 60)

if da:
    print(f'{int(da)} days {int(hrs)} hours {int(mins)} minutes {int(secs)} seconds elapsed')
elif hrs:
    print(f'{int(hrs)} hours {int(mins)} minutes {int(secs)} seconds elapsed')
elif mins:
    print(f'{int(mins)} minutes {int(secs)} seconds elapsed')
else:
    print(f'{int(secs)} seconds elapsed')

<IPython.core.display.Javascript object>

FFMpegWriter is available
Saving frame 1 of 1096
Saving frame 2 of 1096
Saving frame 3 of 1096
Saving frame 4 of 1096
Saving frame 5 of 1096
Saving frame 6 of 1096
Saving frame 7 of 1096
Saving frame 8 of 1096
Saving frame 9 of 1096
Saving frame 10 of 1096
Saving frame 11 of 1096
Saving frame 12 of 1096
Saving frame 13 of 1096
Saving frame 14 of 1096
Saving frame 15 of 1096
Saving frame 16 of 1096
Saving frame 17 of 1096
Saving frame 18 of 1096
Saving frame 19 of 1096
Saving frame 20 of 1096
Saving frame 21 of 1096
Saving frame 22 of 1096
Saving frame 23 of 1096
Saving frame 24 of 1096
Saving frame 25 of 1096
Saving frame 26 of 1096
Saving frame 27 of 1096
Saving frame 28 of 1096
Saving frame 29 of 1096
Saving frame 30 of 1096
Saving frame 31 of 1096
Saving frame 32 of 1096
Saving frame 33 of 1096
Saving frame 34 of 1096
Saving frame 35 of 1096
Saving frame 36 of 1096
Saving frame 37 of 1096
Saving frame 38 of 1096
Saving frame 39 of 1096
Saving frame 40 of 1096
Saving frame 41 of 1096

Saving frame 333 of 1096
Saving frame 334 of 1096
Saving frame 335 of 1096
Saving frame 336 of 1096
Saving frame 337 of 1096
Saving frame 338 of 1096
Saving frame 339 of 1096
Saving frame 340 of 1096
Saving frame 341 of 1096
Saving frame 342 of 1096
Saving frame 343 of 1096
Saving frame 344 of 1096
Saving frame 345 of 1096
Saving frame 346 of 1096
Saving frame 347 of 1096
Saving frame 348 of 1096
Saving frame 349 of 1096
Saving frame 350 of 1096
Saving frame 351 of 1096
Saving frame 352 of 1096
Saving frame 353 of 1096
Saving frame 354 of 1096
Saving frame 355 of 1096
Saving frame 356 of 1096
Saving frame 357 of 1096
Saving frame 358 of 1096
Saving frame 359 of 1096
Saving frame 360 of 1096
Saving frame 361 of 1096
Saving frame 362 of 1096
Saving frame 363 of 1096
Saving frame 364 of 1096
Saving frame 365 of 1096
Saving frame 366 of 1096
Saving frame 367 of 1096
Saving frame 368 of 1096
Saving frame 369 of 1096
Saving frame 370 of 1096
Saving frame 371 of 1096
Saving frame 372 of 1096


Saving frame 662 of 1096
Saving frame 663 of 1096
Saving frame 664 of 1096
Saving frame 665 of 1096
Saving frame 666 of 1096
Saving frame 667 of 1096
Saving frame 668 of 1096
Saving frame 669 of 1096
Saving frame 670 of 1096
Saving frame 671 of 1096
Saving frame 672 of 1096
Saving frame 673 of 1096
Saving frame 674 of 1096
Saving frame 675 of 1096
Saving frame 676 of 1096
Saving frame 677 of 1096
Saving frame 678 of 1096
Saving frame 679 of 1096
Saving frame 680 of 1096
Saving frame 681 of 1096
Saving frame 682 of 1096
Saving frame 683 of 1096
Saving frame 684 of 1096
Saving frame 685 of 1096
Saving frame 686 of 1096
Saving frame 687 of 1096
Saving frame 688 of 1096
Saving frame 689 of 1096
Saving frame 690 of 1096
Saving frame 691 of 1096
Saving frame 692 of 1096
Saving frame 693 of 1096
Saving frame 694 of 1096
Saving frame 695 of 1096
Saving frame 696 of 1096
Saving frame 697 of 1096
Saving frame 698 of 1096
Saving frame 699 of 1096
Saving frame 700 of 1096
Saving frame 701 of 1096


Saving frame 990 of 1096
Saving frame 991 of 1096
Saving frame 992 of 1096
Saving frame 993 of 1096
Saving frame 994 of 1096
Saving frame 995 of 1096
Saving frame 996 of 1096
Saving frame 997 of 1096
Saving frame 998 of 1096
Saving frame 999 of 1096
Saving frame 1000 of 1096
Saving frame 1001 of 1096
Saving frame 1002 of 1096
Saving frame 1003 of 1096
Saving frame 1004 of 1096
Saving frame 1005 of 1096
Saving frame 1006 of 1096
Saving frame 1007 of 1096
Saving frame 1008 of 1096
Saving frame 1009 of 1096
Saving frame 1010 of 1096
Saving frame 1011 of 1096
Saving frame 1012 of 1096
Saving frame 1013 of 1096
Saving frame 1014 of 1096
Saving frame 1015 of 1096
Saving frame 1016 of 1096
Saving frame 1017 of 1096
Saving frame 1018 of 1096
Saving frame 1019 of 1096
Saving frame 1020 of 1096
Saving frame 1021 of 1096
Saving frame 1022 of 1096
Saving frame 1023 of 1096
Saving frame 1024 of 1096
Saving frame 1025 of 1096
Saving frame 1026 of 1096
Saving frame 1027 of 1096
Saving frame 1028 of 1

In [4]:
events

array([(   70, '2020-09-09', 'Mars begins retrograde'),
       (  851, '2020-10-12', 'Mars at opposition'),
       ( 1633, '2020-11-14', 'Mars ends retrograde'),
       (18804, '2022-10-30', 'Mars begins retrograde'),
       (19696, '2022-12-06', 'Mars at opposition'),
       (20588, '2023-01-12', 'Mars ends retrograde')],
      dtype=[('Frame', '<i4'), ('Date', '<U10'), ('Text', '<U25')])